# anything about human body detect
<!--TOC-->
- [头部姿态估计](#头部姿态估计)
  - [Hopenet](#Hopenet)
  - [HyperFace](#HyperFace)
<!--/TOC-->

## 头部姿态估计 (Head Pose Estimation) 
<font style="color:red" size=4> 比较经典的Head Pose Estimation 算法的步骤一般为：
    - 2D人脸关键点检测
    - 3D人脸模型匹配
    - 求解3D点和对应2D点的转换关系
    - 根据旋转矩阵求解欧拉角
</font>

<font style="color:blue" size=3>
 原理：众所周知一个物体相对于相机的姿态可以使用旋转矩阵和平移矩阵来表示。  
    平移矩阵：物体相对于相机的空间位置关系矩阵，用T表示；  
    旋转矩阵：物体相对于相机的空间姿态关系矩阵，用R表示；  
    3D和2D的对应关系如图：  
    <img src="3Dconnect2D.png", width=320, heigth=240>
</font>

### 应用场景和目的
1. 注意力检测：通过判断头部姿态可以判断人的注意力情况，司机，学生
2. 行为分析：通过视频监控分析来分析人的动向、等等
3. 人际互动：头部动作有时可以表示意义，传递信息。(摇头否认，点头同意等等)
4. 实现追踪：也可称为眼球跟踪。实现追踪可以用在游戏领域，

### 如何理解原理
**要先了解相机标定:** 在图像测量过程以及机器视觉应用中，为确定空间物体表面某点的三维几何位置与其在图像中对应点之间的相互关系，必须建立相机成像的几何模型，这些几何模型参数就是相机参数。在大多数条件下这些参数必须通过实验与计算才能得到，这个求解参数的过程就称之为相机标定（或摄像机标定）。无论是在图像测量或者机器视觉应用中，相机参数的标定都是非常关键的环节，其标定结果的精度及算法的稳定性直接影响相机工作产生结果的准确性。因此，做好相机标定是做好后续工作的前提，提高标定精度是科研工作的重点所在。
**相机标定相关问题**
1. 坐标系的转换
  - 世界坐标系
  - 相机坐标系
  - 世界坐标系转换为相机坐标系
  - 像素坐标系、图像坐标系
  - 针孔成像原理
  - 世界坐标系转换为像素坐标系
2. 相机内参与畸变参数
  - 相机内参
  - 畸变参数：径向畸变、切向畸变
  - opencv中的畸变模型
  - 张氏标定中的畸变模型
  - Tasi畸变模型
3. 相机标定

